# Getting Started

In this tutorial, we'll learn how to simulate probabilities with ``pathfinder2e_stats``.

To follow this tutorial, you'll need to have at least a basic understanding of
- The [Pathfinder rules](https://2e.aonprd.com/PlayersGuide.aspx), and
- Data science workflows, e.g. based on Python + pandas + Jupyter notebooks. See {ref}`audience`.

If you don't have your Jupyter Notebook development environment ready yet, go back to {doc}`../installing`.

## Rolling some dice
Let's start simple - let's import the module and roll a d6.

We're going to roll it *one hundred thousand times.*

In [ ]:
import pathfinder2e_stats as pf2

oned6 = pf2.roll(1, 6)
oned6

``pathfinder2e_stats`` functions return standard {class}`xarray.DataArray` and {class}`xarray.Dataset` objects, which can be analyzed with standard data science techniques. We can start immediately answering some questions - for example, what is the mean roll?

In [ ]:
oned6.mean()

Note that the result above is a *numerical approximation*: the mean of rolling 1d6 an *infinite* amount of times is *exactly* 3.5. If we roll it less times than that, however, there's going to be some error.

If we roll it again, we are going to get a different sequence. This is because ``pathfinder2e_stats`` uses a global random number generator, which by default is reset to a fixed seed every time you restart your notebook. See {func}`~pathfinder2e_stats.rng` and {func}`~pathfinder2e_stats.seed`.

In [ ]:
pf2.roll(1, 6)

Well, that was easy, but we could have figured out the answer by doing the maths on the back of an envelope! Let's move on to something that is more complicated. A timeless classic: a 6d6 {prd_spells}`Fireball <1530>`!

In [ ]:
fireball = pf2.roll(6, 6)
fireball

What is the damage distribution? First we're going to calculate it numerically; then we'll visualize it with ``matplotlib`` (but we could use any other library, like ``plotly`` or ``hvplot``).

In [ ]:
fireball.value_counts("roll").to_pandas()

In [ ]:
_ = fireball.to_pandas().hist(bins=30)

But wait - that's just the base damage! The *actual* damage of a fireball depends on the target's reflex saving throw, as well as their resistances, immunities and weaknesses. ``pathfinder2e_stats`` makes dealing with all this very easy.

## Rolling checks

TODO

## Rolling damage

TODO

## Conditional buffs/debuffs

TODO

## Next steps

Congratulations, you finished the basic tutorial!

From here, you can go look at the {doc}`index`.
In the {doc}`../api`, you will find many functions, flags and options that were omitted here for the sake of brevity.